In [13]:
#Handle Imports
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import json
import pprint
import io
import requests
from PIL import Image, ImageDraw, ImageFilter, ImageChops
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import uuid
from ollama import chat
from ollama import ChatResponse
from random import randrange
import os
import glob
import shutil
import textwrap
from datetime import datetime
import dropbox

#load from .env file which should hopefully exist
load_dotenv()

#initialize spotipy client
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [2]:
#given an image performs the processing
def clear_images():
   for hgx in glob.glob("/Users/MurphyD1/Documents/Python/sloppa/images/*"):
        #print(hgx)
        os.remove(hgx)
   for hgx in glob.glob("/Users/MurphyD1/Documents/Python/sloppa/final_images/*"):
        #print(hgx)
        os.remove(hgx)
    
def slop_generator(track, artist, star):
    response: ChatResponse = chat(model='llama3.2', messages=[
      {
        'role': 'user',
        'content': f"""Generate some vague but smart-sounding commentary on the track {track} by {artist}. Should be indicative of a {star} star review. 20 words max and should sound like it was written by a relatively young person. Each review should end with some stupid catchphrase that's 4-5 words long and only generate one response please. each output should be 20 words max""",
      },
    ])
    return response['message']['content']


def download_image(link):
    url = link
    response = requests.get(url)
    file_path = str(uuid.uuid4()) + ".jpg"
    if response.status_code == 200:
        with open("images/" + file_path, 'wb') as file:
            file.write(response.content)
    return file_path


#given a playlist builds a dictionary of items 
def load_items(playlist):
    track_list = []
    result = sp.playlist(playlist, fields=None, market=None, additional_types=('track',))
    tracks = result['tracks']['items']
    tracks = tracks

    for track in tracks:
        
        #extract track information 
        track = track['track']
        name = track['name']
        artists = track['artists']
        image = track['album']['images'][0]['url']
        artist  = (artists[0]['name'])
        
        #download image
        res = download_image(image)

        #assign random star rating and generate text slop for it
        stars = randrange(1,5)
        slop = slop_generator(name, artist, stars)

        #print(name, artist, image)
        d = {
            "name": name,
            "artist": artist,
            "image": image,
            "image_id": res,
            "analysis": slop,
            "stars": stars
        }
        track_list.append(d)

        #res = download_image(image)
    return track_list  

In [3]:
#given an image performs the processing
#this could still use a lot of cleaning up harcoded values
def process_item(obj, path):
    STAR_SIZE = 65
    CENTER_IMG_SIZE = 1080
    TT_WIDTH = 1080
    TT_HEIGHT = 1920
    TITLE_SIZE = 62
    SUBTITLE_SIZE = 52
    PARAGRAPH_SIZE = 45
    TITLE_DIVIDER_SIZE = 31
    STAR_DIVIDER_SIZE = 75
    
    #Gather File Information
    track_title = obj['name']
    track_artist = obj['artist']
    image_id = obj['image_id']
    stars = obj['stars']
    slop = obj['analysis']
    file_path = f"""images/{image_id}"""
    
    #open and adjust star images
    star_f = Image.open('fill.png')
    star_u = Image.open('unfilled.png')
    star_f = star_f.resize((STAR_SIZE,STAR_SIZE), Image.LANCZOS)
    star_u = star_u.resize((STAR_SIZE,STAR_SIZE), Image.LANCZOS)

    #open and resize track image
    original_img = Image.open(file_path)
    resized_img = original_img.resize((CENTER_IMG_SIZE,CENTER_IMG_SIZE), Image.LANCZOS)
    
    #create a new blank image
    new_width, new_height = TT_WIDTH,TT_HEIGHT
    color = (166,123,91)
    background = Image.new("RGBA", (new_width, new_height), color)
    
    #paste track image onto background
    x_offset = 0
    y_offset = (TT_HEIGHT - TT_WIDTH) // 2
    background.paste(resized_img, (x_offset, y_offset))

    #image will be saved in PNG format since it's transparent, higher quality.
    png_format = image_id.replace(".jpg", ".png")
    new_filename = f"""{path}/{png_format}"""

    #initialize draw object to paste stars/text
    draw = ImageDraw.Draw(background)

    #load required fonts
    title_font = ImageFont.truetype("SourceCodePro-Bold.ttf",TITLE_SIZE)
    subtitle_font = ImageFont.truetype("SourceCodePro-SemiBold.ttf",SUBTITLE_SIZE)
    paragraph_font = ImageFont.truetype("SourceCodePro-Medium.ttf", PARAGRAPH_SIZE)
    
    title_text = f"""{track_artist}"""
    subtitle_text = f"""{track_title}"""
    paragraph_text = slop

    # text_file.write(track_artist + " : " + track_title + ":" + "\n")
    # text_file.write(slop[1:-1])
    # text_file.write("\n")
    # text_file.write("\n")

    n_width = TT_WIDTH
    n_height = (TT_HEIGHT - CENTER_IMG_SIZE)/2
    n_height_b = CENTER_IMG_SIZE + n_height

    
    draw.text((n_width/2, n_height/2 - TITLE_DIVIDER_SIZE), title_text, font=title_font, anchor="mm", fill=(0,0,0,255))
    draw.text((n_width/2, n_height/2 + TITLE_DIVIDER_SIZE), subtitle_text, font=subtitle_font, anchor="mm", fill=(0,0,0,255))

    h3 = int(n_height/2 + STAR_DIVIDER_SIZE)
    w3  = int(n_width/2 - STAR_SIZE * 2.5)
    for x in range(0,5):
        if x < stars:
            background.paste(star_f, (w3, h3), mask=star_f) 
        else:
            background.paste(star_u, (w3, h3), mask=star_f) 
        w3 = w3 + 65

    blurred = Image.new('RGBA', background.size)
    font = ImageFont.truetype('SourceCodePro-Bold.ttf', 85)

    #write out individual lines from the generated text
    lines = textwrap.wrap(paragraph_text, width=20)
    
    number_lines = len(lines)
    draw2 = ImageDraw.Draw(blurred)

    starting_height = TT_HEIGHT//2 - (number_lines // 2 * 60)
    for line in lines:
        #draw2.text(xy=(TT_WIDTH//2,starting_height), text=line, fill='black', font=font, anchor='mm')
        draw2.text(xy=(TT_WIDTH//2,starting_height), text=line, fill='black', font=font, anchor='mm')

        starting_height = starting_height + 60

    blurred = blurred.filter(ImageFilter.BoxBlur(7))


    background.paste(blurred,blurred)

    starting_height = TT_HEIGHT//2 - (number_lines // 2 * 60)

    for line in lines:
        draw.text(xy=(TT_WIDTH//2,starting_height), text=line, fill='white', font=font, anchor='mm')
        starting_height = starting_height + 60

    
    background.save(new_filename)

    background.close()
    original_img.close()
    star_f.close()
    star_u.close()


In [4]:
items = load_items('spotify:playlist:2EV1gXRFByVNFfGwjBEYTI')
#clear_images()

In [9]:
len(items)

71

In [16]:
j = 1
batch_path = []
for i in range(0,len(items),5):
    batch = items[i:i+5]
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    j = j + 1

    print(timestamp + f"""-{j}""")


    newpath = f"""final_images/{timestamp}-{j}"""
    batch_path.append(newpath)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for obj in batch:
        process_item(obj, newpath)


# #clear_images()
# timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# newpath = r'C:\Program Files\arbitrary' 
# if not os.path.exists(newpath):
#     os.makedirs(newpath)


# items = items[:5]
# print(len(items))
# text_file = open("final_images/outputs.txt", "a")
# text_file.truncate(0)

# for item in items:
#     process_item(item, text_file)

# text_file.close()


2025-04-14_21-54-21-2
2025-04-14_21-54-22-3
2025-04-14_21-54-23-4
2025-04-14_21-54-24-5
2025-04-14_21-54-24-6
2025-04-14_21-54-25-7
2025-04-14_21-54-26-8
2025-04-14_21-54-27-9
2025-04-14_21-54-28-10
2025-04-14_21-54-29-11
2025-04-14_21-54-30-12
2025-04-14_21-54-31-13
2025-04-14_21-54-32-14
2025-04-14_21-54-33-15
2025-04-14_21-54-34-16


In [14]:
def send_slop_folder(pwd):
    SOURCE_DIR = f"""/Users/MurphyD1/Documents/Python/sloppa/{pwd}"""  # Local folder with images/text files
    ACCESS_TOKEN ="sl.u.AFq_twT-UDgbTMLp7xS9yiyIkIlN9vuGuQ8hfABQBTORtAbxD6t1zUkY6BpiUpEmI6bjOWyuCfcKSOhR5QZ7yWhY-IPEtzKm_kFtojwzmLczYQvci4mpOZW2MyfWouht6ZSGMB_Gg_KUXkv0c8L78T7uGuCgGLBjng-ysa5reYaw8JvbiFxbkYqXRHaHORJm1OiV6pbdbi9B13uWUYyCX32erUuIOJxZLBO2mE0Ce6caqjMpz28MZ1N5xEmnwHQoNJRf7f3jHAR2TgbqgZ8tzDjBt7p-cRadFYhNAz7wdWo3PYsQRXGsGQeneFB9Sis8VS-wvn7BteCmp9jJJbfCU22pbZKzXNDXkchcO23wqjB78iVkOGwRVzNC47ntZUcNdfsYcwqylRagHMXjhaLPhCb_o_x-v5NKCRq2aphr1wWzY1zIEPdzu0h74LD_zZ49rsGWUACqeuaYO9L0TZcLogpuVOy0y-uKGAlbWJsBVBnax35o4_FGqQWI8lzUBzWPLg1PzOGvpvfIYjX01_xG1hrZRbv7WFGZHHcN96-BbvYfIrwctgXeT3ZxPED2R-K2-G4RXkR328qDtILkFacbSf9jNxa96EL7_ScJO1NJGEaN0-9heD1F3-00SDinkpjfVFcVS8ZiCiU2MiSwvZXqm_n2XyeFBh3ZV6Y7vvHpdAu62OM-8uvnanJ1smmYZJcPoYBSPKhYef2pPyWAYuH2R-vr6qVjO3w3ZCCSX0XDKYwxn9oVMSKiyWnHyFBb85F4k3G7jS1P6vXCGa6YkdSlhQA-Vuo4ne58f2X34SkKs_shDxETGchbrsR105iNd73EOIHtA7-tPmYFmxfBrz8aalj_6oGZTrVH6-Y4r7CyxqoGLwIStC5NzkXoB_9Ks90_1RRq32w6BTtpC3tLkWHuHs-ncxCgte-_BXziRwFBgAru8vc8mfHAShAIMBGc5_sbVR4PqBfOS72WeA4-kGUUFMWa_-cVUrIiH6T6eYS5gQbo0uBPbKfeCINYIlHrLj8qXKik6Wa7K2lYhut-SlHsgdCpJKrKuzHiPxvSp9WXooe7VhVYW_xZzRSg1EvYclt86dO6hwszx4jEdG7-fO20NYdN4pZmDljTmTL0AJaPUEiuFZc-5IRrinDvI0dOqrAQzSoRsAh2nhas0d8elA5TBs_tjqTNj2gTtFZEpJiKTwGCzJAEUTRy24Xa-w3Q50D5s4ja27kqjB95OboJYF1wdJ6koGBJiLeMRZajgdP6dQb-bBOd5PIqrKqzkN564xytcNVUpmCfHlYCW9eauP-yH0_P"
    TARGET_FOLDER = '/slop'  # Base Dropbox folder
    ALLOWED_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.txt'}
    # Format timestamp for subfolder
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
    dropbox_subfolder = f"{TARGET_FOLDER}/{timestamp}"
    ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
    print(ACCESS_TOKEN)
    dbx = dropbox.Dropbox(ACCESS_TOKEN)
    # Create Dropbox subfolder
    try:
        dbx.files_create_folder_v2(dropbox_subfolder)
        print(f"Created Dropbox folder: {dropbox_subfolder}")
    except dropbox.exceptions.ApiError as e:
        if e.error.is_path() and e.error.get_path().is_conflict():
            print("Folder already exists.")
        else:
            raise
    # Upload all allowed files from the directory
    for filename in os.listdir(SOURCE_DIR):
        local_path = os.path.join(SOURCE_DIR, filename)
        _, ext = os.path.splitext(filename)
        if os.path.isfile(local_path) and ext.lower() in ALLOWED_EXTENSIONS:
            with open(local_path, 'rb') as f:
                dropbox_path = f"{dropbox_subfolder}/{filename}"
                dbx.files_upload(f.read(), dropbox_path)
                print(f"Uploaded {filename} to {dropbox_path}")

In [18]:
#batch_path
for path in batch_path:
    send_slop_folder(path)


sl.u.AFq_twT-UDgbTMLp7xS9yiyIkIlN9vuGuQ8hfABQBTORtAbxD6t1zUkY6BpiUpEmI6bjOWyuCfcKSOhR5QZ7yWhY-IPEtzKm_kFtojwzmLczYQvci4mpOZW2MyfWouht6ZSGMB_Gg_KUXkv0c8L78T7uGuCgGLBjng-ysa5reYaw8JvbiFxbkYqXRHaHORJm1OiV6pbdbi9B13uWUYyCX32erUuIOJxZLBO2mE0Ce6caqjMpz28MZ1N5xEmnwHQoNJRf7f3jHAR2TgbqgZ8tzDjBt7p-cRadFYhNAz7wdWo3PYsQRXGsGQeneFB9Sis8VS-wvn7BteCmp9jJJbfCU22pbZKzXNDXkchcO23wqjB78iVkOGwRVzNC47ntZUcNdfsYcwqylRagHMXjhaLPhCb_o_x-v5NKCRq2aphr1wWzY1zIEPdzu0h74LD_zZ49rsGWUACqeuaYO9L0TZcLogpuVOy0y-uKGAlbWJsBVBnax35o4_FGqQWI8lzUBzWPLg1PzOGvpvfIYjX01_xG1hrZRbv7WFGZHHcN96-BbvYfIrwctgXeT3ZxPED2R-K2-G4RXkR328qDtILkFacbSf9jNxa96EL7_ScJO1NJGEaN0-9heD1F3-00SDinkpjfVFcVS8ZiCiU2MiSwvZXqm_n2XyeFBh3ZV6Y7vvHpdAu62OM-8uvnanJ1smmYZJcPoYBSPKhYef2pPyWAYuH2R-vr6qVjO3w3ZCCSX0XDKYwxn9oVMSKiyWnHyFBb85F4k3G7jS1P6vXCGa6YkdSlhQA-Vuo4ne58f2X34SkKs_shDxETGchbrsR105iNd73EOIHtA7-tPmYFmxfBrz8aalj_6oGZTrVH6-Y4r7CyxqoGLwIStC5NzkXoB_9Ks90_1RRq32w6BTtpC3tLkWHuHs-ncxCgte-_BXziRwFBgAru8vc8mfHAShAIMBGc5_sbVR4PqBfOS72WeA4-kGUUFMWa_-cVUrIiH6T

ReadTimeout: HTTPSConnectionPool(host='content.dropboxapi.com', port=443): Read timed out. (read timeout=100)